In [1]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC

In [2]:
df = pd.read_csv('wine.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       178 non-null    float64
 1   malic_acid                    178 non-null    float64
 2   ash                           178 non-null    float64
 3   alcalinity_of_ash             178 non-null    float64
 4   magnesium                     178 non-null    float64
 5   total_phenols                 178 non-null    float64
 6   flavanoids                    178 non-null    float64
 7   nonflavanoid_phenols          178 non-null    float64
 8   proanthocyanins               178 non-null    float64
 9   color_intensity               178 non-null    float64
 10  hue                           178 non-null    float64
 11  od280/od315_of_diluted_wines  178 non-null    float64
 12  proline                       178 non-null    float64
 13  label

In [3]:
data = df.drop('label', axis=1)
label = df['label']

In [5]:
train_d, test_d, train_l, test_l = train_test_split(data, label, test_size=0.3, random_state=0)

In [7]:
param_grid = {'C': [1, 10, 100, 1000, 1e4, 1e5, 1e6, 1e7, 1e8], 'gamma': [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 0.1, 1]}
clf = GridSearchCV(SVC(), param_grid, cv=5)
clf.fit(train_d, train_l)
res = pd.DataFrame(clf.cv_results_)
res.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.004415,0.001269,0.002682,0.000975,1,1e-08,"{'C': 1, 'gamma': 1e-08}",0.40,0.40,0.40,0.40,0.375000,0.395000,0.010000,72
1,0.003322,0.000498,0.002174,0.000901,1,1e-07,"{'C': 1, 'gamma': 1e-07}",0.64,0.64,0.72,0.72,0.541667,0.652333,0.065892,58
2,0.002837,0.000054,0.001820,0.000063,1,1e-06,"{'C': 1, 'gamma': 1e-06}",0.64,0.64,0.72,0.72,0.541667,0.652333,0.065892,58
3,0.002852,0.000118,0.001769,0.000146,1,1e-05,"{'C': 1, 'gamma': 1e-05}",0.64,0.68,0.76,0.76,0.666667,0.701333,0.049603,43
4,0.003119,0.000261,0.001747,0.000051,1,0.0001,"{'C': 1, 'gamma': 0.0001}",0.64,0.64,0.76,0.76,0.625000,0.685000,0.061482,44


In [9]:
res.sort_values('rank_test_score').head(3)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
64,0.024143,0.007274,0.002312,0.000404,1e+07,1e-07,"{'C': 10000000.0, 'gamma': 1e-07}",0.88,0.96,1.00,0.96,0.958333,0.951667,0.039129,1
55,0.020729,0.008592,0.001977,0.000105,1e+06,1e-07,"{'C': 1000000.0, 'gamma': 1e-07}",0.88,0.96,0.96,1.00,0.958333,0.951667,0.039129,1
73,0.024963,0.004371,0.001961,0.000074,1e+08,1e-07,"{'C': 100000000.0, 'gamma': 1e-07}",0.88,0.96,1.00,0.96,0.958333,0.951667,0.039129,1


In [11]:
res[res['rank_test_score'] == 1].params

55      {'C': 1000000.0, 'gamma': 1e-07}
64     {'C': 10000000.0, 'gamma': 1e-07}
73    {'C': 100000000.0, 'gamma': 1e-07}
Name: params, dtype: object

In [16]:
C_list = res[res['rank_test_score'] == 1].param_C.values
gamma_list = res[res['rank_test_score'] == 1].param_gamma.values
score_list = res[res['rank_test_score'] == 1].mean_test_score.values

In [19]:
clf1 = SVC(C=C_list[0], gamma=gamma_list[0])
clf2 = SVC(C=C_list[1], gamma=gamma_list[1])
clf3 = SVC(C=C_list[2], gamma=gamma_list[2])

clf1.fit(train_d, train_l)
clf2.fit(train_d, train_l)
clf3.fit(train_d, train_l)
score1 = clf1.score(test_d, test_l)
score2 = clf2.score(test_d, test_l)
score3 = clf3.score(test_d, test_l)

print(f"Param(1): C={C_list[0]}\tgamma={gamma_list[0]}\tScore: {score1}")
print(f"Param(2): C={C_list[1]}\tgamma={gamma_list[1]}\tScore: {score2}")
print(f"Param(3): C={C_list[2]}\tgamma={gamma_list[2]}\tScore: {score3}")

Param(1): C=1000000.0	gamma=1e-07	Score: 1.0
Param(2): C=10000000.0	gamma=1e-07	Score: 0.9444444444444444
Param(3): C=100000000.0	gamma=1e-07	Score: 0.9444444444444444
